<a href="https://colab.research.google.com/github/lisichkinpavel/recommendations_system/blob/lesson2/rec_dz2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 5.5MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406396 sha256=5ac009f4319320ed6ddd751f3992077df3abb01868d1f81df26bc049ac29068e
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Notebooks/recomendation_systems/2/webinar_2/data/retail_train.csv')
data.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
data.shape

(2396804, 12)

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

Может я неправильно понял как нужно делать сэмплы пропорционально весу.
В np.random.choice есть параметр р который соответствует весам элементов списка.Сумма списка должна = 1 (что логично). Я только не понял насчет логарифма. Если взять логарифм от количества купленного товара, то значение будет от 0 до 1. Что потом делать с этими значениями? Отскейлить?

Я попытался сделать немного по-другому

In [6]:
# Посчитаем сколько раз покупался кадый товар
item_weights = data.groupby('item_id')['item_id'].agg('count').reset_index(name="count")
item_weights.head(5)

,item_id,count
0,25671,3
1,26081,1
2,26093,1
3,26190,1
4,26355,1


In [7]:
# Теперь присвоим итемам вес согласно количеству покупок(разделим количество купленного товара на общее количество покупок)
item_weights['weight'] = item_weights['count'].apply(lambda x: x / np.sum(item_weights['count']))

In [8]:
item_weights.sort_values(by=['weight'], ascending=False)

,item_id,count,weight
35054,1082185,27362,1.141604e-02
56233,6534178,18364,7.661870e-03
29195,1029743,13455,5.613726e-03
25333,995242,11397,4.755082e-03
37719,1106523,9175,3.828014e-03
...,...,...,...
60249,7442882,1,4.172223e-07
60250,7442885,1,4.172223e-07
60251,7442888,1,4.172223e-07
29677,1034166,1,4.172223e-07


In [9]:
sum(item_weights['weight'])

0.9999999999997984

In [10]:
item_weights = item_weights.drop(columns='count')
item_weights.head(3)

,item_id,weight
0,25671,1.251667e-06
1,26081,4.172223e-07
2,26093,4.172223e-07


In [11]:
def weighted_random_recommendation(items_weights:pd.DataFrame, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code

    items = item_weights.item_id
    weights = item_weights.weight
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [12]:
%%time

# your_code
weighted_random_recommendation(item_weights)

CPU times: user 3.83 ms, sys: 0 ns, total: 3.83 ms
Wall time: 5.66 ms


[1105623, 9527494, 6534178, 6391504, 1133654]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [29]:
result = pd.read_csv('/content/drive/MyDrive/Notebooks/recomendation_systems/2/webinar_2/webinar_2/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


Тут почему-то фичипредставлены не как списки,а как строки, нужно перевести в нормальный вид.

In [30]:
type(result.own_purchases[0])

str

In [31]:
columns = result.iloc[:,1:].columns.tolist()
columns

['actual',
 'random_recommendation',
 'popular_recommendation',
 'itemitem',
 'cosine',
 'tfidf',
 'own_purchases']

In [32]:
for i in columns:
  result[i] = result[i].apply(lambda x:list(map(int,x.strip("[],").replace(',','').split())))

In [33]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [34]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [45]:
# Словарь для посчитанных метрик
metrics = {'random_recommendation':[],
 'popular_recommendation':[],
 'itemitem':[],
 'cosine':[],
 'tfidf':[]}

In [46]:
metrics

{'cosine': [],
 'itemitem': [],
 'popular_recommendation': [],
 'random_recommendation': [],
 'tfidf': []}

In [47]:
# Считаем метрики каждого типа рекомендаций для каждого покупателя
for rec in recomendation_types:
  for i in range(result.shape[0]):
    metrics[rec].append(precision_at_k(result[rec][i], result['actual'][i], k=5))


In [54]:
# Посмотрим среднее значение метрик по всем покупателям
for key, value in metrics.items():
  print(f'{key} : {np.mean(value)}')

random_recommendation : 0.0005876591576885408
popular_recommendation : 0.15523996082272282
itemitem : 0.03359451518119491
cosine : 0.03525954946131244
tfidf : 0.036141038197845254


Видим что рекомендации популярных товаров работают лучше остальных. Самый низкий результат показывают случайные рекомендации.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code